In [1]:
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import osmnx as ox
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_al=pd.read_excel("C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/#RAW/Data/FullSUPIdentifiedData.xlsx")


In [3]:
df_all=df_al[['stationid', 'Latitude', 'Longitude']]
df_all=df_all.drop_duplicates(subset=['stationid'], keep='first')
df_all.reset_index(drop=True, inplace=True)

In [4]:
gdf_all=gpd.GeoDataFrame(df_all, geometry=gpd.points_from_xy(df_all.Longitude, df_all.Latitude), crs='EPSG:4326')
gdf_all.to_crs(epsg=2277, inplace=True)

#### Preparation

In [72]:
import os
files= os.listdir('Data/weather/Texas GSOY station data')
weather_files=[f for f in files if f.endswith('.csv')]
dataset=[]
for i in weather_files:
    file=pd.read_csv("Data/weather/Texas GSOY station data/"+i)
    dataset.append(file)
weather_data=pd.concat(dataset)

In [73]:
#Info on all weather stations in texas (thousands..)
file="Data/weather/Texas GSOY station data/stations_info.txt"
df = pd.read_csv(file, sep='\s+', header=None, usecols=[0,1,2], names=['stationid', 'Lat', 'Lon'])


In [74]:
weather_data2=weather_data.merge(df, right_on="stationid", left_on="STATION")
weather_data3=weather_data2[['stationid','DATE','Lon','Lat','PRCP', 'TAVG','TMAX', 'TMIN', 'AWND']]
weather_data3=weather_data3.rename(columns={"stationid":"weather_station"})
weather_data3['DATE']=(weather_data3['DATE'].astype(str).str[:4]).astype(int)

In [75]:
from math import radians, sin, cos, sqrt, atan2
df_all2=df_all.copy(deep=True)
weather_data_unq=weather_data3.drop_duplicates(subset = ['weather_station'], keep='first')
#Haversine formula
def calc_distance(lon1, lat1, lon2, lat2):
    R = 6371  # earth radius in km
    dlon = radians(lon2 - lon1)
    dlat = radians(lat2 - lat1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

for i, row in df_all.iterrows():
    # calculate the distance between this station and all stations in the second dataset
    distances = []
    for j, row2 in weather_data_unq.iterrows(): 
        distance = calc_distance(row['Longitude'], row['Latitude'], row2['Lon'], row2['Lat'])
        distances.append((row2['weather_station'], distance, row['stationid']))
    # find the STATIONID in the second dataset with the shortest distance
    min_distance = min(distances, key=lambda x: x[1])
    distances.remove(min_distance)
    min_distance2 = min(distances, key=lambda x: x[1])
    distances.remove(min_distance2)
    min_distance3 = min(distances, key=lambda x: x[1])
    distances.remove(min_distance3)
    min_distance4 = min(distances, key=lambda x: x[1])

    #print(min_distance2)
    # assign the STATIONID to the corresponding row in the first dataset
    df_all2.loc[i, 'weather_station'] = min_distance[0]
    df_all2.loc[i, 'distance(km)'] = min_distance[1]
    df_all2.loc[i, 'weather_station2'] = min_distance2[0]
    df_all2.loc[i, 'distance(km)2'] = min_distance2[1]
    df_all2.loc[i, 'weather_station3'] = min_distance3[0]
    df_all2.loc[i, 'distance(km)3'] = min_distance3[1]
    df_all2.loc[i, 'weather_station4'] = min_distance4[0]
    df_all2.loc[i, 'distance(km)4'] = min_distance4[1]


In [76]:
df_all2.to_csv("Data/Temp/FULL_nearest_weather_stationid_v2.csv")


Selecting stations with > 10KM distance

In [77]:
df_all3=df_all2.copy(deep=True)
df_all3.set_index(df_all3.stationid, drop=True, inplace=True)
df_all3.loc[(df_all3['distance(km)4']>10), 'weather_station4']=np.nan #only weather_station4 is checked because others were checked manually

In [78]:
weather_data4=weather_data3.copy(deep=True)
#weather_data4.set_index(weather_data3.weather_station, drop=True, inplace=True)

In [79]:
df_aa=pd.read_excel("C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/#RAW/Data/Temp/FULL_prepared_data_ampm_v7.xlsx")
#df_aa=df_aa[['stationid', 'year']]

In [80]:
df_stations=df_aa.merge(df_all3, left_on=df_aa.stationid, right_on=df_all3.stationid,)
df_stations.rename(columns={'stationid_x':'stationid'}, inplace=True)
df_stations.drop(['stationid_y', 'key_0', 'Latitude', 'Longitude'], axis=1, inplace=True)

In [ ]:
#Joining weather data and stations based on year
a=df_stations.merge(weather_data4, right_on=['weather_station', 'DATE'], left_on=['weather_station', 'year'], how='left')
b=a.groupby(['stationid','year'],as_index=False )[('DATE',	'PRCP',	'TAVG',	'TMAX',	'TMIN',	'AWND')].first()
ws1 = b.rename(columns={c: c+'_1' for c in b.columns if c not in ['stationid', 'year']})

a=df_stations.merge(weather_data4, right_on=['weather_station', 'DATE'], left_on=['weather_station2', 'year'], how='left')
b=a.groupby(['stationid','year'],as_index=False )[('DATE',	'PRCP',	'TAVG',	'TMAX',	'TMIN',	'AWND')].first()
ws2 = b.rename(columns={c: c+'_2' for c in b.columns if c not in ['stationid', 'year']})

a=df_stations.merge(weather_data4, right_on=['weather_station', 'DATE'], left_on=['weather_station3', 'year'], how='left')
b=a.groupby(['stationid','year'],as_index=False )[('DATE',	'PRCP',	'TAVG',	'TMAX',	'TMIN',	'AWND')].first()
ws3 = b.rename(columns={c: c+'_3' for c in b.columns if c not in ['stationid', 'year']})

a=df_stations.merge(weather_data4, right_on=['weather_station', 'DATE'], left_on=['weather_station4', 'year'], how='left')
b=a.groupby(['stationid','year'],as_index=False )[('DATE',	'PRCP',	'TAVG',	'TMAX',	'TMIN',	'AWND')].first()
ws4 = b.rename(columns={c: c+'_4' for c in b.columns if c not in ['stationid', 'year']})


In [ ]:
#Getting missing values from the next nearest weather station
#PRCP
ws1.loc[ws1.PRCP_1.isna(), 'PRCP_1']= ws2.loc[ws1.PRCP_1.isna(), 'PRCP_2']	
ws1.loc[ws1.PRCP_1.isna(), 'PRCP_1']= ws3.loc[ws1.PRCP_1.isna(), 'PRCP_3']	
ws1.loc[ws1.PRCP_1.isna(), 'PRCP_1']= ws4.loc[ws1.PRCP_1.isna(), 'PRCP_4']	
#TAVG
ws1.loc[ws1.TAVG_1.isna(), 'TAVG_1']= ws2.loc[ws1.TAVG_1.isna(), 'TAVG_2']	
ws1.loc[ws1.TAVG_1.isna(), 'TAVG_1']= ws3.loc[ws1.TAVG_1.isna(), 'TAVG_3']	
ws1.loc[ws1.TAVG_1.isna(), 'TAVG_1']= ws4.loc[ws1.TAVG_1.isna(), 'TAVG_4']	
#TMAX
ws1.loc[ws1.TMAX_1.isna(), 'TMAX_1']= ws2.loc[ws1.TMAX_1.isna(), 'TMAX_2']	
ws1.loc[ws1.TMAX_1.isna(), 'TMAX_1']= ws3.loc[ws1.TMAX_1.isna(), 'TMAX_3']	
ws1.loc[ws1.TMAX_1.isna(), 'TMAX_1']= ws4.loc[ws1.TMAX_1.isna(), 'TMAX_4']	
#TMIN
ws1.loc[ws1.TMIN_1.isna(), 'TMIN_1']= ws2.loc[ws1.TMIN_1.isna(), 'TMIN_2']	
ws1.loc[ws1.TMIN_1.isna(), 'TMIN_1']= ws3.loc[ws1.TMIN_1.isna(), 'TMIN_3']	
ws1.loc[ws1.TMIN_1.isna(), 'TMIN_1']= ws4.loc[ws1.TMIN_1.isna(), 'TMIN_4']	
#AWND
ws1.loc[ws1.AWND_1.isna(), 'AWND_1']= ws2.loc[ws1.AWND_1.isna(), 'AWND_2']	
ws1.loc[ws1.AWND_1.isna(), 'AWND_1']= ws3.loc[ws1.AWND_1.isna(), 'AWND_3']	
ws1.loc[ws1.AWND_1.isna(), 'AWND_1']= ws4.loc[ws1.AWND_1.isna(), 'AWND_4']	


In [83]:
f = {
    'PRCP_1': np.mean,
    'TAVG_1': np.mean,
    'TMAX_1': np.mean,
    'TMIN_1': np.mean,
    'AWND_1': np.mean
}
grouped = ws1.groupby(['stationid']).agg(f) #finding mean for FIllna
merged = ws1.merge(grouped, on='stationid', suffixes=('_ws1', '_grouped'))
# Fill the null values in the merged dataset with the values from 'grouped'
merged['PRCP_1_ws1'].fillna(merged['PRCP_1_grouped'], inplace=True)
merged['TAVG_1_ws1'].fillna(merged['TAVG_1_grouped'], inplace=True)
merged['TMAX_1_ws1'].fillna(merged['TMAX_1_grouped'], inplace=True)
merged['TMIN_1_ws1'].fillna(merged['TMIN_1_grouped'], inplace=True)
merged['AWND_1_ws1'].fillna(merged['AWND_1_grouped'], inplace=True)

# Drop the columns with '_grouped' suffix
merged.drop(['PRCP_1_grouped', 'TAVG_1_grouped', 'TMAX_1_grouped', 'TMIN_1_grouped', 'AWND_1_grouped'], axis=1, inplace=True)
DF=merged.rename(columns={'PRCP_1_ws1':'PRCP',
                       'TAVG_1_ws1':'TAVG',
                       'TMAX_1_ws1':'TMAX',
                       'TMIN_1_ws1':'TMIN',
                       'AWND_1_ws1':'AWND',})
DF.drop(['DATE_1'], axis=1, inplace=True)

In [86]:
DF.to_csv("Data/Temp/FULL_nearest_weather_data_v4.csv")

#### Distance to nearest water and  campus

In [5]:
gdf_all=gpd.GeoDataFrame(df_all, geometry=gpd.points_from_xy(df_all.Longitude, df_all.Latitude), crs='EPSG:4326')
# gdf_all.to_crs(epsg=2277, inplace=True)

In [6]:
import osmnx as ox
import pandas as pd

# get water body data using OpenStreetMap
place_name = "Texas"
tags = {"natural": "water"}
water = ox.geometries_from_place(place_name, tags)
water_geometry = water['geometry'].unary_union

In [ ]:
water.to_crs(epsg=2277, inplace=True)
gdf_all.to_crs(epsg=2277, inplace=True)

In [ ]:
from shapely.geometry import Point


# assume your data is in GeoDataFrames called `gdf_all` and `water`
# extract the geometry of the water bodies
water_geometry = water['geometry'].unary_union

# define a function to calculate the minimum distance between a point and the water bodies
def min_distance_to_water(point, water_geometry):
    return point.distance(water_geometry)

# calculate the distance for each point
gdf_all['proximity_water'] = gdf_all.geometry.apply(min_distance_to_water, water_geometry=water_geometry)


In [ ]:
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
#VISUALIZE
from shapely.geometry import Polygon, MultiPolygon, LineString
import contextily as ctx

# Find the nearest water body to the station
distances = water['geometry'].apply(lambda x: x.distance(gdf_all.geometry.iloc[0]))
nearest_water = distances.idxmin()
nearest_water_geom = water.loc[nearest_water].geometry

station_point = gdf_all.geometry.iloc[0]

if isinstance(nearest_water_geom, (Polygon, MultiPolygon)):
    nearest_water_geom = nearest_water_geom.boundary

# Calculate the nearest point on the water body to the station
nearest_point = nearest_water_geom.interpolate(nearest_water_geom.project(station_point))

# Create a LineString connecting the station and the nearest point on the water body
line = LineString([station_point, nearest_point])

# Create a GeoDataFrame for the line
line_gdf = gpd.GeoDataFrame(geometry=[line])

# Plot the data

ax = water.plot(color='blue', alpha=0.5, figsize=(10, 10))
#shp_road17_.plot(ax=ax, color="grey")
gdf_all.iloc[[0]].plot(ax=ax, color='maroon', markersize=80)
line_gdf.plot(ax=ax, color='yellow', linestyle='--', linewidth=2)
# ax.set_xlim(2887000,2892000)
# ax.set_ylim(9735000,9740000)
ctx.add_basemap(ax=ax, crs=gdf_all.crs.to_string(),source=cx.providers.Stamen.TonerLite)
ax.set_xlim(3162000,3170000)
ax.set_ylim(10094500,10102000)

# Create legend
legend_elements = [  Patch(facecolor='grey', edgecolor='black', label='Road Network'),    
                   Patch(facecolor="#800000", edgecolor='black', label='Count station'),    
                   Patch(facecolor='blue', edgecolor='black', label='Water body'),    
                   Line2D([0], [0], color='yellow', lw=2, linestyle='--', label='Shortest path')]
plt.legend(handles=legend_elements, title='Legend', loc='lower right')

ax.set_xticks([]) 
ax.set_yticks([]) 
plt.show()


NameError: name 'water' is not defined

In [20]:
shp_road17_=gpd.read_file("C:/Users/P-Koirala/OneDrive - Texas A&M Transportation Institute/Desktop/Data/SUP/#RAW/Data/Txdot_roadway/2017/TxDOT_Roadway_Linework_wAssets.shp")
shp_road17_.to_crs(epsg=2277, inplace=True)

KeyboardInterrupt: 

##### Campus

In [19]:
gdf_all2=gdf_all.copy(deep=True)

In [20]:
state = 'Texas'
amenity = ['university', 'community_college']

# Query OpenStreetMap to get the universities in Texas
query = f'amenity={amenity} and addr:state={state}'
uni = ox.geometries_from_place(state, tags={'amenity': amenity}, which_result=None)

In [21]:
uni.to_crs(epsg=2277, inplace=True)

In [22]:
uni_geometry = uni['geometry'].unary_union
def min_distanc(point, uni_geometry):
    return point.distance(uni_geometry)

gdf_all2['proximity_uni'] = gdf_all2.geometry.apply(min_distanc, uni_geometry=uni_geometry)

In [23]:
gdf_all2=gdf_all2.rename(columns={'distance_to_water':'proximity_water'})

In [24]:
#gdf_all2.to_csv("Data/Temp/FULL_distance_uni_data_.csv")

##### School

In [25]:
state = 'Texas'
amenity = ['school']

# Query OpenStreetMap to get the universities in Texas
query = f'amenity={amenity} and addr:state={state}'
sco = ox.geometries_from_place(state, tags={'amenity': amenity}, which_result=None)

In [26]:
uni.to_crs(epsg=2277, inplace=True)

In [27]:
sco_geometry = sco['geometry'].unary_union
def min_distanc(point, sco_geometry):
    return point.distance(sco_geometry)

gdf_all2['proximity_school'] = gdf_all2.geometry.apply(min_distanc, sco_geometry=sco_geometry)

In [28]:
gdf_all2.to_csv("Data/Temp/FULL_proximity_data_v3.csv")

#### parks

In [29]:
place_name="Texas"
tags = {'leisure': 'park', 'landuse': 'grass'}
parks = ox.geometries_from_place(place_name, tags)

In [30]:
parks.to_crs(epsg=2277, inplace=True)

In [31]:
parks_geometry = parks['geometry'].unary_union
def min_distanc(point, parks_geometry):
    return point.distance(parks_geometry)

gdf_all2['proximity_parks'] = gdf_all2.geometry.apply(min_distanc, parks_geometry=parks_geometry)

In [32]:
gdf_all2.to_csv("Data/Temp/trial.csv")

In [33]:
gdf_all2.to_csv("Data/Temp/FULL_proximity_data_v4.csv")